In [59]:
import pandas as pd
import numpy as np

df = pd.read_csv('ncr_ride_bookings.csv')
df.shape, df.head()

((150000, 21),
          Date      Time    Booking ID   Booking Status   Customer ID  \
 0  2024-03-23  12:29:38  "CNR5884300"  No Driver Found  "CID1982111"   
 1  2024-11-29  18:01:39  "CNR1326809"       Incomplete  "CID4604802"   
 2  2024-08-23  08:56:10  "CNR8494506"        Completed  "CID9202816"   
 3  2024-10-21  17:17:25  "CNR8906825"        Completed  "CID2610914"   
 4  2024-09-16  22:08:00  "CNR1950162"        Completed  "CID9933542"   
 
     Vehicle Type      Pickup Location      Drop Location  Avg VTAT  Avg CTAT  \
 0          eBike          Palam Vihar            Jhilmil       NaN       NaN   
 1       Go Sedan        Shastri Nagar  Gurgaon Sector 56       4.9      14.0   
 2           Auto              Khandsa      Malviya Nagar      13.4      25.8   
 3  Premier Sedan  Central Secretariat           Inderlok      13.1      28.5   
 4           Bike     Ghitorni Village        Khan Market       5.3      19.6   
 
    ...  Reason for cancelling by Customer Cancelled Ride

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Date                               150000 non-null  object 
 1   Time                               150000 non-null  object 
 2   Booking ID                         150000 non-null  object 
 3   Booking Status                     150000 non-null  object 
 4   Customer ID                        150000 non-null  object 
 5   Vehicle Type                       150000 non-null  object 
 6   Pickup Location                    150000 non-null  object 
 7   Drop Location                      150000 non-null  object 
 8   Avg VTAT                           139500 non-null  float64
 9   Avg CTAT                           102000 non-null  float64
 10  Cancelled Rides by Customer        10500 non-null   float64
 11  Reason for cancelling by Customer  1050

In [61]:
# convert to datetime
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
# create datetime column by combining Date and Time
df['datetime'] = df['Date'] + pd.to_timedelta(df['Time'])
# validate
df[['Date', 'Time', 'datetime']].isna().sum()

Date        0
Time        0
datetime    0
dtype: int64

In [62]:
# checking consistencies
df['is_completed'] = df['Booking Status'].eq('Completed').astype(int)
df['is_cancelled'] = (
    (df['Cancelled Rides by Customer'] == 1) |
    (df['Cancelled Rides by Driver'] == 1)
).astype(int)

df.query("is_completed == 1 and is_cancelled == 1").shape

(0, 24)

In [63]:
invalid_ratings = df[
    (df['is_completed'] == 0) &
    ((df['Customer Rating'].notna()) | (df['Driver Ratings'].notna()))
]

invalid_ratings.shape

df.loc[df['is_completed'] == 0, ['Customer Rating', 'Driver Ratings']] = np.nan

In [64]:
df.isna().mean().sort_values(ascending=False)

Incomplete Rides Reason              0.94
Incomplete Rides                     0.94
Cancelled Rides by Customer          0.93
Reason for cancelling by Customer    0.93
Cancelled Rides by Driver            0.82
Driver Cancellation Reason           0.82
Customer Rating                      0.38
Driver Ratings                       0.38
Payment Method                       0.32
Avg CTAT                             0.32
Ride Distance                        0.32
Booking Value                        0.32
Avg VTAT                             0.07
Booking ID                           0.00
Time                                 0.00
Date                                 0.00
Drop Location                        0.00
Pickup Location                      0.00
Vehicle Type                         0.00
Customer ID                          0.00
Booking Status                       0.00
datetime                             0.00
is_completed                         0.00
is_cancelled                      

In [65]:
# feature engineering
# time based
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.day_name()
df['is_weekend'] = df['datetime'].dt.weekday.isin([5,6]).astype(int)

# cancellation type
df['cancellation_type'] = np.select(
    [
        df['Cancelled Rides by Customer'] == 1,
        df['Cancelled Rides by Driver'] == 1
    ],
    [
        'Customer',
        'Driver'
    ],
    default='None'
)

# revenue per km
df['revenue_per_km'] = np.where(
    df['Ride Distance'] > 0,
    df['Booking Value'] / df['Ride Distance'],
    np.nan
)

# trip duration category
df['trip_duration_category'] = pd.cut(
    df['Avg CTAT'],
    bins=[0, 15, 30, 60, np.inf],
    labels=['Short', 'Medium', 'Long', 'Very Long']
)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 30 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   Date                               150000 non-null  datetime64[ns]
 1   Time                               150000 non-null  object        
 2   Booking ID                         150000 non-null  object        
 3   Booking Status                     150000 non-null  object        
 4   Customer ID                        150000 non-null  object        
 5   Vehicle Type                       150000 non-null  object        
 6   Pickup Location                    150000 non-null  object        
 7   Drop Location                      150000 non-null  object        
 8   Avg VTAT                           139500 non-null  float64       
 9   Avg CTAT                           102000 non-null  float64       
 10  Cancelled Rides by C

In [ ]:
df.to_csv("data_cleaned.csv", index=False)